In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline, RectBivariateSpline

# Astronomy
import astropy
from astropy.units import au, minute

from typing import List, Optional

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_pos
from asteroid_spline import get_df_shape, make_spline_df, spline_ast_data
from planets_interp import get_earth_pos
from asteroid_direction import calc_dir_linear, calc_dir_linear_topos
from asteroid_direction_test import jpl_ast_dir_populate
from ra_dec import radec2dir
from ra_dec_test import direction_diff
from db_utils import sp2df, df2db
import matplotlib.pyplot as plt

# Test Astrometric Direction with JPL State Vectors

In [3]:
# Import the JPL asteroid directions
sp_name = 'JPL.AstrometricDirectionTest'
df = sp2df(sp_name=sp_name)

In [4]:
df

,AsteroidID,TimeID,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,vAst_x,vAst_y,vAst_z,LightTime,tObs,ux,uy,uz,delta
0,1,69120000,48000.0,-0.880028,-0.483807,-0.000050,-1.155150,2.297484,0.282873,-0.009457,-0.005464,0.001583,23.363621,48000.0,-0.097885,0.990090,0.100704,2.809226
1,1,69127200,48005.0,-0.836785,-0.557668,-0.000054,-1.202187,2.269674,0.290725,-0.009357,-0.005659,0.001558,23.833511,48005.0,-0.127458,0.986641,0.101460,2.865725
2,1,69134400,48010.0,-0.787422,-0.627462,-0.000056,-1.248709,2.240893,0.298453,-0.009251,-0.005853,0.001533,24.289747,48010.0,-0.157895,0.982153,0.102200,2.920583
3,1,69141600,48015.0,-0.732357,-0.692671,-0.000054,-1.294694,2.211151,0.306052,-0.009142,-0.006044,0.001507,24.731042,48015.0,-0.189059,0.976556,0.102932,2.973644
4,1,69148800,48020.0,-0.672042,-0.752868,-0.000054,-1.340122,2.180457,0.313519,-0.009028,-0.006233,0.001480,25.156619,48020.0,-0.220819,0.969791,0.103659,3.024815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48011,16,90691200,62980.0,-0.842921,-0.549267,0.000119,-0.813070,2.724700,-0.105745,-0.010217,-0.001563,0.000351,27.244467,62980.0,0.009167,0.999436,-0.032318,3.275856
48012,16,90698400,62985.0,-0.794209,-0.619483,0.000126,-0.864021,2.716450,-0.103974,-0.010163,-0.001737,0.000357,27.764006,62985.0,-0.020859,0.999296,-0.031185,3.338325
48013,16,90705600,62990.0,-0.739772,-0.685171,0.000131,-0.914696,2.707332,-0.102170,-0.010107,-0.001910,0.000364,28.265182,62990.0,-0.051416,0.998224,-0.030103,3.398587
48014,16,90712800,62995.0,-0.680044,-0.745908,0.000132,-0.965081,2.697356,-0.100334,-0.010047,-0.002080,0.000370,28.747137,62995.0,-0.082410,0.996174,-0.029068,3.456536


In [5]:
# Column names
cols_q_obs = ['qObs_x', 'qObs_y', 'qObs_z']
cols_q_ast = ['qAst_x', 'qAst_y', 'qAst_z']
cols_v_ast = ['vAst_x', 'vAst_y', 'vAst_z']
cols_u = ['ux', 'uy', 'uz']

In [6]:
# Position and velocity according to JPL
q_obs = df[cols_q_obs].values
q_ast = df[cols_q_ast].values
v_ast = df[cols_v_ast].values

# Direction according to JPL
u_jpl = df[cols_u].values

In [7]:
# MSE calculation of the astrometric direction in linear model
u_mse, delta_mse = calc_dir_linear(q_tgt=q_ast, v_tgt=v_ast, q_obs=q_obs)

In [8]:
direction_diff(name1='JPL', name2='MSE', u1=u_jpl, u2=u_mse, verbose=True)

Angle Difference: MSE vs. JPL
Mean  :   0.000237 deg (   0.851 seconds)
Median:   0.000248 deg (   0.892 seconds)
Max   :   0.000518 deg (   1.864 seconds)


0.8514511996886399

# Test Astrometric Direction with MSE State Vectors

In [9]:
# Directions from two different sources
df_jpl = sp2df(sp_name='JPL.AstrometricDirectionTest')
df_mse = sp2df(sp_name='JPL.AstrometricDirectionAndVectorTest')

In [10]:
# State vectors according to JPL integration
q_obs_jpl = df_jpl[cols_q_obs].values
q_ast_jpl = df_jpl[cols_q_ast].values
v_ast_jpl = df_jpl[cols_v_ast].values

In [11]:
# State vectors according to MSE integration
q_obs_mse = df_mse[cols_q_obs].values
q_ast_mse = df_mse[cols_q_ast].values
v_ast_mse = df_mse[cols_v_ast].values

In [12]:
# Directions calculated in linear model with both state vectors
u_jpl = calc_dir_linear(q_tgt=q_ast_jpl, v_tgt=v_ast_jpl, q_obs=q_obs_jpl)
u_mse = calc_dir_linear(q_tgt=q_ast_mse, v_tgt=v_ast_mse, q_obs=q_obs_mse)

In [13]:
# Mask both frames down to overlapping dates (multiples of 20)
mask_interval = 20*1440
mask_jpl = (df_jpl.TimeID % mask_interval) == 0
df_jpl = df_jpl[mask_jpl].reindex()

mask_mse = (df_mse.TimeID % mask_interval) == 0
df_mse = df_mse[mask_mse].reindex()

In [14]:
df_jpl

,AsteroidID,TimeID,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,vAst_x,vAst_y,vAst_z,LightTime,tObs,ux,uy,uz,delta
0,1,69120000,48000.0,-0.880028,-0.483807,-0.000050,-1.155150,2.297484,0.282873,-0.009457,-0.005464,0.001583,23.363621,48000.0,-0.097885,0.990090,0.100704,2.809226
4,1,69148800,48020.0,-0.672042,-0.752868,-0.000054,-1.340122,2.180457,0.313519,-0.009028,-0.006233,0.001480,25.156619,48020.0,-0.220819,0.969791,0.103659,3.024815
8,1,69177600,48040.0,-0.387421,-0.936160,-0.000062,-1.515844,2.048398,0.342000,-0.008533,-0.006966,0.001366,26.689323,48040.0,-0.351587,0.930068,0.106584,3.209106
12,1,69206400,48060.0,-0.059117,-1.013904,-0.000068,-1.681013,1.902098,0.368097,-0.007974,-0.007656,0.001242,27.919420,48060.0,-0.483095,0.868673,0.109664,3.357012
16,1,69235200,48080.0,0.275917,-0.978167,-0.000064,-1.834396,1.742481,0.391613,-0.007355,-0.008297,0.001108,28.820901,48080.0,-0.608928,0.785133,0.113019,3.465406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47999,16,90604800,62920.0,-0.888357,0.439177,0.000041,-0.185713,2.752974,-0.124104,-0.010617,0.000657,0.000257,20.137859,62920.0,0.290246,0.955578,-0.051272,2.421363
48003,16,90633600,62940.0,-0.989707,0.109228,0.000069,-0.397338,2.758441,-0.118616,-0.010536,-0.000107,0.000291,22.598897,62940.0,0.218060,0.974958,-0.043679,2.717276
48007,16,90662400,62960.0,-0.973271,-0.233761,0.000098,-0.606806,2.748854,-0.112479,-0.010402,-0.000848,0.000322,25.010115,62960.0,0.121919,0.991834,-0.037437,3.007199
48011,16,90691200,62980.0,-0.842921,-0.549267,0.000119,-0.813070,2.724700,-0.105745,-0.010217,-0.001563,0.000351,27.244467,62980.0,0.009167,0.999436,-0.032318,3.275856


# Test Astrometric Direction with MSE State Vectors and Topos

In [15]:
df_mse

,AsteroidID,TimeID,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,vAst_x,vAst_y,vAst_z,LightTime,tObs,ux,uy,uz,delta
0,1,69120000,48000.0,-0.880033,-0.483797,-0.000050,-1.155142,2.297488,0.282872,-0.009457,-0.005464,0.001583,23.363621,48000.0,-0.097885,0.990090,0.100704,2.809226
1,1,69148800,48020.0,-0.672050,-0.752861,-0.000054,-1.340115,2.180462,0.313518,-0.009028,-0.006233,0.001480,25.156619,48020.0,-0.220819,0.969791,0.103659,3.024815
2,1,69177600,48040.0,-0.387430,-0.936156,-0.000062,-1.515837,2.048404,0.341999,-0.008533,-0.006966,0.001366,26.689323,48040.0,-0.351587,0.930068,0.106584,3.209106
3,1,69206400,48060.0,-0.059127,-1.013903,-0.000068,-1.681006,1.902104,0.368096,-0.007974,-0.007656,0.001242,27.919420,48060.0,-0.483095,0.868673,0.109664,3.357012
4,1,69235200,48080.0,0.275907,-0.978170,-0.000064,-1.834390,1.742487,0.391612,-0.007355,-0.008297,0.001108,28.820901,48080.0,-0.608928,0.785133,0.113019,3.465406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12011,16,90604800,62920.0,-0.888358,0.439174,0.000041,-0.185713,2.752974,-0.124104,-0.010617,0.000657,0.000257,20.137859,62920.0,0.290246,0.955578,-0.051272,2.421363
12012,16,90633600,62940.0,-0.989708,0.109224,0.000069,-0.397338,2.758441,-0.118616,-0.010536,-0.000107,0.000291,22.598897,62940.0,0.218060,0.974958,-0.043679,2.717276
12013,16,90662400,62960.0,-0.973270,-0.233765,0.000098,-0.606806,2.748854,-0.112479,-0.010402,-0.000848,0.000322,25.010115,62960.0,0.121919,0.991834,-0.037437,3.007199
12014,16,90691200,62980.0,-0.842919,-0.549270,0.000119,-0.813071,2.724700,-0.105745,-0.010217,-0.001563,0.000351,27.244467,62980.0,0.009167,0.999436,-0.032318,3.275856


# Test asteroid direction with iterated splines

In [16]:
# Import the JPL asteroid directions
df = sp2df(sp_name='JPL.AstrometricDirectionTest')

In [17]:
# The Horizons DataFrame
df

,AsteroidID,TimeID,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,vAst_x,vAst_y,vAst_z,LightTime,tObs,ux,uy,uz,delta
0,1,69120000,48000.0,-0.880028,-0.483807,-0.000050,-1.155150,2.297484,0.282873,-0.009457,-0.005464,0.001583,23.363621,48000.0,-0.097885,0.990090,0.100704,2.809226
1,1,69127200,48005.0,-0.836785,-0.557668,-0.000054,-1.202187,2.269674,0.290725,-0.009357,-0.005659,0.001558,23.833511,48005.0,-0.127458,0.986641,0.101460,2.865725
2,1,69134400,48010.0,-0.787422,-0.627462,-0.000056,-1.248709,2.240893,0.298453,-0.009251,-0.005853,0.001533,24.289747,48010.0,-0.157895,0.982153,0.102200,2.920583
3,1,69141600,48015.0,-0.732357,-0.692671,-0.000054,-1.294694,2.211151,0.306052,-0.009142,-0.006044,0.001507,24.731042,48015.0,-0.189059,0.976556,0.102932,2.973644
4,1,69148800,48020.0,-0.672042,-0.752868,-0.000054,-1.340122,2.180457,0.313519,-0.009028,-0.006233,0.001480,25.156619,48020.0,-0.220819,0.969791,0.103659,3.024815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48011,16,90691200,62980.0,-0.842921,-0.549267,0.000119,-0.813070,2.724700,-0.105745,-0.010217,-0.001563,0.000351,27.244467,62980.0,0.009167,0.999436,-0.032318,3.275856
48012,16,90698400,62985.0,-0.794209,-0.619483,0.000126,-0.864021,2.716450,-0.103974,-0.010163,-0.001737,0.000357,27.764006,62985.0,-0.020859,0.999296,-0.031185,3.338325
48013,16,90705600,62990.0,-0.739772,-0.685171,0.000131,-0.914696,2.707332,-0.102170,-0.010107,-0.001910,0.000364,28.265182,62990.0,-0.051416,0.998224,-0.030103,3.398587
48014,16,90712800,62995.0,-0.680044,-0.745908,0.000132,-0.965081,2.697356,-0.100334,-0.010047,-0.002080,0.000370,28.747137,62995.0,-0.082410,0.996174,-0.029068,3.456536


In [18]:
from asteroid_direction import light_time_iter, calc_dir_spline

In [19]:
# Extract arrays from Horizons DataFrame
q_obs = df[cols_q_ast].values
t_obs = df.tObs.values
asteroid_id = df.AsteroidID.values
light_time = df.LightTime.values
u_jpl = df[cols_u].values

In [20]:
# Initial light time - save a copy
lt0 = light_time.copy()

In [21]:
# Build asteroid position spline
id_col = 'AsteroidID'
time_col = 'tAst'
spline_q_ast = make_spline_df(df=df, cols_spline=cols_q_ast, id_col=id_col, time_col=time_col)

In [25]:
np.mean(np.abs(light_time - lt0))

22.569418127158254

In [24]:
# One iteration of light time refinement
light_time = light_time_iter(spline_q_ast=spline_q_ast, q_obs=q_obs, t_obs=t_obs, asteroid_id=asteroid_id, light_time=light_time)

In [ ]:
calc_dir_spline(spline_q_ast: spline_type_ast, q_obs: np.ndarray, t_obs: np.ndarray, 
                    asteroid_id: np.ndarray, light_time: np.ndarray, iters:int=2)

In [13]:
# Calculate the direction and light time
df = calc_dir_ast2obs(n0=n0, n1=n1)
# Mask down to Ceres
mask = df.AsteroidID==1
dfm = df[mask]
# Take every 5th row to get data every 20 days
dfm = dfm.iloc[::5]
dfm.reset_index(inplace=True)

In [14]:
# MSE data frame for asteroid directions
dfm

,index,AsteroidID,TimeID,tAst,qAst_x,qAst_y,qAst_z,tObs,qObs_x,qObs_y,qObs_z,LightTime,ux,uy,uz
0,0,1,69120000,47999.983776,-1.155142,2.297488,0.282872,48000.0,-0.880033,-0.483797,-0.000050,23.362923,-0.097934,0.990084,0.100715
1,5,1,69148800,48019.982530,-1.339957,2.180571,0.313492,48020.0,-0.672050,-0.752861,-0.000054,25.156518,-0.220867,0.969778,0.103669
2,10,1,69177600,48039.981466,-1.515679,2.048533,0.341973,48040.0,-0.387430,-0.936156,-0.000062,26.689240,-0.351634,0.930049,0.106593
3,15,1,69206400,48059.980612,-1.680852,1.902253,0.368072,48060.0,-0.059128,-1.013903,-0.000068,27.919357,-0.483140,0.868647,0.109672
4,20,1,69235200,48079.979986,-1.834242,1.742653,0.391590,48080.0,0.275907,-0.978170,-0.000064,28.820858,-0.608968,0.785100,0.113026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,3730,1,90604800,62919.983586,1.040245,2.523575,-0.111216,62920.0,-0.888358,0.439174,0.000041,23.635688,0.678583,0.733480,-0.039139
747,3735,1,90633600,62939.982228,0.841699,2.580912,-0.072849,62940.0,-0.989708,0.109224,0.000069,25.591547,0.595127,0.803283,-0.023686
748,3740,1,90662400,62959.981056,0.638171,2.622996,-0.034051,62960.0,-0.973270,-0.233765,0.000098,27.279676,0.491230,0.870968,-0.010400
749,3745,1,90691200,62979.980116,0.430794,2.649302,0.004953,62980.0,-0.842919,-0.549270,0.000119,28.633330,0.369905,0.929069,0.001415


In [15]:
# Get test DataFrame from JPL
sp_name = 'JPL.AsteroidDirectionTest'
dfh = sp2df(sp_name=sp_name)

# Take every 4th row to get data every 20 days; this matches exactly on MSE integration points
dfh = dfh.iloc[::4]
dfh.reset_index(inplace=True)

In [16]:
dfh

,index,AsteroidID,TimeID,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,...,vAst_y,vAst_z,LightTime,tObs,ux_ast,uy_ast,uz_ast,ux_app,uy_app,uz_app
0,0,1,69120000,47999.983775,-0.880028,-0.483807,-0.000050,-1.155150,2.297484,0.282873,...,-0.005464,0.001583,23.363621,48000.0,-0.097885,0.990090,0.100704,-0.095560,0.990312,0.100744
1,4,1,69134400,48009.983132,-0.787422,-0.627462,-0.000056,-1.248709,2.240893,0.298453,...,-0.005853,0.001533,24.289747,48010.0,-0.157895,0.982153,0.102200,-0.155585,0.982517,0.102239
2,8,1,69148800,48019.982530,-0.672042,-0.752868,-0.000054,-1.340122,2.180457,0.313519,...,-0.006233,0.001480,25.156619,48020.0,-0.220819,0.969791,0.103659,-0.218535,0.970304,0.103696
3,12,1,69163200,48029.981973,-0.537449,-0.856722,-0.000062,-1.429222,2.116260,0.328044,...,-0.006605,0.001424,25.958353,48030.0,-0.285668,0.952547,0.105113,-0.283425,0.953212,0.105149
4,16,1,69177600,48039.981466,-0.387421,-0.936160,-0.000062,-1.515844,2.048398,0.342000,...,-0.006966,0.001366,26.689323,48040.0,-0.351587,0.930068,0.106584,-0.349400,0.930888,0.106617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24003,96012,16,90655200,62954.983042,-0.988478,-0.149078,0.000090,-0.554700,2.752636,-0.114071,...,-0.000665,0.000315,24.419855,62955.0,0.147790,0.988254,-0.038882,0.140214,0.989357,-0.038904
24004,96016,16,90669600,62964.982230,-0.950906,-0.316688,0.000103,-0.658711,2.744160,-0.110849,...,-0.001029,0.000330,25.589057,62965.0,0.095022,0.994822,-0.036062,0.087406,0.995519,-0.036085
24005,96020,16,90684000,62974.981453,-0.885496,-0.475018,0.000111,-0.761859,2.732075,-0.107481,...,-0.001387,0.000344,26.707754,62975.0,0.038555,0.998695,-0.033506,0.030914,0.998960,-0.033529
24006,96024,16,90698400,62984.980719,-0.794209,-0.619483,0.000126,-0.864021,2.716450,-0.103974,...,-0.001737,0.000357,27.764006,62985.0,-0.020859,0.999296,-0.031185,-0.028499,0.999106,-0.031210


In [ ]:
# Relevant columns for astrom